I am building a model to predict When can the Toronto Police expect a surge of crime in Toronto (Day of week, Time of Day and Month ) based on the Toronto police crime data set. 
To do this, I would be exploring 3 models 

1. Time Series Forecasting Models(Xgboost): I am chosing this model because crime occurance often show temporal dependencies which is that they current trends and this model is good at detecting such patterns and seasonality.

2. Ensemble Learning Models: This model helps to capture non-linear relationships among features for both categorical and numeric data.  

3. Bayesian models: Incorporates uncertainty by using probabilty which is good when dealing with limited data 

First model we are building is the time series forcasting model. 
1. The first thing we need is the trend of crime occurance 
2. The second thing is seasonality. This is the repeated pattern of data over a set period in time. Eg the way crime occurs during maybe the holiday season, or weekends, or summer periods
3. Cycle is the non-seasonal patterns in the data. This could be occurance during maybe the close of down of the pandemic or economic downturn 
4. Variations are unpredictable up and down that can not be explained by the other variables above 

Models examples
ARIMA : Auto regressive integrated moving average : This looks at how past values affect future values through trends, seasonality and cycles while removing the noise (Variations that dont make sense) 

Expoential smoothing : This is used for data that has no trend, seasonality or cycle but only has variations. It works by giving more weight to recent value and less weight to previous values

Implementation 


In [2]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [121]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np


In [151]:
crime_data = pd.read_csv('https://raw.githubusercontent.com/Pelumioluwa/MBAN6110Class/main/Data/Toronto%20Police%20Shooting%20and%20Firearm%20Discharge.csv')
crime_data

,X,Y,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,...,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.234955,43.781528,1,GO-2004397105,2004/05/15 04:00:00+00,2004,May,Saturday,136,15,...,Night,D41,0,0,142,Woburn North,137,Woburn (137),-79.234955,43.781528
1,-79.618218,43.733547,2,GO-2004600109,2004/07/26 04:00:00+00,2004,July,Monday,208,26,...,Evening,D23,0,0,1,West Humber-Clairville,1,West Humber-Clairville (1),-79.618218,43.733547
2,-79.518759,43.769157,3,GO-2004311851,2004/05/14 04:00:00+00,2004,May,Friday,135,14,...,Afternoon,D31,1,1,24,Black Creek,24,Black Creek (24),-79.518759,43.769157
3,-79.378870,43.660665,4,GO-2004736004,2004/12/19 05:00:00+00,2004,December,Sunday,354,19,...,Night,D51,1,0,168,Downtown Yonge East,75,Church-Yonge Corridor (75),-79.378870,43.660665
4,-79.212435,43.812075,5,GO-2004303455,2004/04/12 04:00:00+00,2004,April,Monday,103,12,...,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.212435,43.812075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,-79.362941,43.771764,5703,GO-20221038418,2022/06/02 04:00:00+00,2022,June,Thursday,153,2,...,Night,D33,0,1,53,Henry Farm,53,Henry Farm (53),-79.362941,43.771764
5703,-79.402702,43.643920,5704,GO-20221362949,2022/07/17 04:00:00+00,2022,July,Sunday,198,17,...,Night,D14,1,1,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.402702,43.643920
5704,-79.196217,43.786621,5705,GO-2022336423,2022/02/21 05:00:00+00,2022,February,Monday,52,21,...,Night,D43,0,0,135,Morningside,135,Morningside (135),-79.196217,43.786621
5705,-79.443993,43.658083,5706,GO-20221765442,2022/09/11 04:00:00+00,2022,September,Sunday,254,11,...,Night,D11,0,2,83,Dufferin Grove,83,Dufferin Grove (83),-79.443993,43.658083


In [152]:
crime_data.shape

(5707, 21)

In [153]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5707 entries, 0 to 5706
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  5707 non-null   float64
 1   Y                  5707 non-null   float64
 2   OBJECTID           5707 non-null   int64  
 3   EVENT_UNIQUE_ID    5707 non-null   object 
 4   OCC_DATE           5707 non-null   object 
 5   OCC_YEAR           5707 non-null   int64  
 6   OCC_MONTH          5707 non-null   object 
 7   OCC_DOW            5707 non-null   object 
 8   OCC_DOY            5707 non-null   int64  
 9   OCC_DAY            5707 non-null   int64  
 10  OCC_HOUR           5707 non-null   int64  
 11  OCC_TIME_RANGE     5707 non-null   object 
 12  DIVISION           5707 non-null   object 
 13  DEATH              5707 non-null   int64  
 14  INJURIES           5707 non-null   int64  
 15  HOOD_158           5707 non-null   object 
 16  NEIGHBOURHOOD_158  5707 

In [119]:
#convert the occurance date to datetime 
#crime_data['OCC_DATE'] = pd.to_datetime(crime_data['OCC_DATE'])
#crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5707 entries, 0 to 5706
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   X                  5707 non-null   float64            
 1   Y                  5707 non-null   float64            
 2   OBJECTID           5707 non-null   int64              
 3   EVENT_UNIQUE_ID    5707 non-null   object             
 4   OCC_DATE           5707 non-null   datetime64[ns, UTC]
 5   OCC_YEAR           5707 non-null   int64              
 6   OCC_MONTH          5707 non-null   object             
 7   OCC_DOW            5707 non-null   object             
 8   OCC_DOY            5707 non-null   int64              
 9   OCC_DAY            5707 non-null   int64              
 10  OCC_HOUR           5707 non-null   int64              
 11  OCC_TIME_RANGE     5707 non-null   object             
 12  DIVISION           5707 non-null   object       

In [125]:
crime_data.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR',
       'OCC_MONTH', 'OCC_DOW', 'OCC_DOY', 'OCC_DAY', 'OCC_HOUR',
       'OCC_TIME_RANGE', 'DIVISION', 'DEATH', 'INJURIES', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84',
       'LAT_WGS84'],
      dtype='object')

Build a pipeline to convert the dataframe 
1. drop unnecessary columns not needed in the - X', 'Y', 'OBJECTID','DIVISION', 'DEATH', 'INJURIES', 'HOOD_158','NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'
2. Convert day of the week, month, day of the week, occurance time range to dummy data and one encode it 
3. Scale the numerical variables (day of the year,day of the month, hour of the day)
4. Make date time the index



In [154]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_MONTH','OCC_DOW','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
label = ['OCC_DOW']
target = ['OCC_DOW']
#label_features = ['OCC_TIME_RANGE','OCC_MONTH','OCC_HOUR','OCC_DOW']


Predict the probability that each day would have a crime , the period of the day and the month and the time of the day)

In [155]:
crime_data[label]

,OCC_DOW
0,Saturday
1,Monday
2,Friday
3,Sunday
4,Monday
...,...
5702,Thursday
5703,Sunday
5704,Monday
5705,Sunday


In [156]:
label_encoder_trans = LabelEncoder()
#crime_data['OCC_TIME_RANGE'] = label_encoder_trans.fit_transform(crime_data['OCC_TIME_RANGE'])
#crime_data['OCC_MONTH'] = label_encoder_trans.fit_transform(crime_data['OCC_MONTH'])
#crime_data['OCC_HOUR'] = label_encoder_trans.fit_transform(crime_data['OCC_HOUR'])
crime_data['OCC_DOW'] = label_encoder_trans.fit_transform(crime_data['OCC_DOW'])
target = crime_data['OCC_DOW']

In [157]:
crime_data[label]

,OCC_DOW
0,2
1,1
2,0
3,3
4,1
...,...
5702,4
5703,3
5704,1
5705,3


In [158]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data['OCC_DOW'],
                                                    test_size = 0.3, random_state=123)

In [159]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [160]:
#building processor
#preprocessor = ColumnTransformer(transformers=[('label', label_encoder_trans, label_features),
#                                               ('categorical', categorical_transformer, categorical_features),
 #                                              ('numerical', numerical_transformer, numerical_features)])
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),('numerical',numerical_transformer,numerical_features) ])

In [161]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder())])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])
#label_encoder_trans = Pipeline(steps=[('label_encoder', LabelEncoder())])


In [162]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5707 entries, 0 to 5706
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  5707 non-null   float64
 1   Y                  5707 non-null   float64
 2   OBJECTID           5707 non-null   int64  
 3   EVENT_UNIQUE_ID    5707 non-null   object 
 4   OCC_DATE           5707 non-null   object 
 5   OCC_YEAR           5707 non-null   int64  
 6   OCC_MONTH          5707 non-null   object 
 7   OCC_DOW            5707 non-null   int64  
 8   OCC_DOY            5707 non-null   int64  
 9   OCC_DAY            5707 non-null   int64  
 10  OCC_HOUR           5707 non-null   int64  
 11  OCC_TIME_RANGE     5707 non-null   object 
 12  DIVISION           5707 non-null   object 
 13  DEATH              5707 non-null   int64  
 14  INJURIES           5707 non-null   int64  
 15  HOOD_158           5707 non-null   object 
 16  NEIGHBOURHOOD_158  5707 

In [163]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [164]:
random_forest.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_MONTH', 'OCC_DOW',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [165]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [166]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

      Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
0       0.00    0.00      0.01    0.00      0.01     0.00       0.98
1       0.00    0.01      0.00    0.00      0.01     0.98       0.00
2       0.99    0.00      0.00    0.00      0.01     0.00       0.00
3       0.00    0.98      0.01    0.00      0.01     0.00       0.00
4       0.00    0.00      0.01    0.99      0.00     0.00       0.00
...      ...     ...       ...     ...       ...      ...        ...
1708    0.00    0.00      0.00    0.00      0.00     1.00       0.00
1709    0.00    0.00      0.00    1.00      0.00     0.00       0.00
1710    0.99    0.00      0.00    0.01      0.00     0.00       0.00
1711    1.00    0.00      0.00    0.00      0.00     0.00       0.00
1712    0.00    0.00      0.00    0.00      0.00     0.01       0.99

[1713 rows x 7 columns]


Further to the predictions, I want to evaluate the accuracy of my model 

In [167]:
#import evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


In [168]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)
print(predicted_day_names)
print(predicted_classes)


['Wednesday' 'Tuesday' 'Friday' ... 'Friday' 'Friday' 'Wednesday']
[6 5 0 ... 0 0 6]


In [169]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [170]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[239   0   0   0   0   0   0]
 [  0 230   0   0   0   0   0]
 [  0   0 290   0   0   0   0]
 [  0   0   0 317   0   0   0]
 [  0   0   0   0 202   0   0]
 [  0   0   0   0   0 224   0]
 [  0   0   0   0   0   0 211]]


In [171]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
classification_rep = classification_report(y_test, predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

      Friday       1.00      1.00      1.00       239
      Monday       1.00      1.00      1.00       230
    Saturday       1.00      1.00      1.00       290
      Sunday       1.00      1.00      1.00       317
    Thursday       1.00      1.00      1.00       202
     Tuesday       1.00      1.00      1.00       224
   Wednesday       1.00      1.00      1.00       211

    accuracy                           1.00      1713
   macro avg       1.00      1.00      1.00      1713
weighted avg       1.00      1.00      1.00      1713



In [172]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 1.0


In [173]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.139615,0.134425,0.169305,0.184577,0.118208,0.130549,0.123322
std,0.342740,0.336876,0.372078,0.385334,0.318180,0.331684,0.323545
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [175]:
#get the day with the highest sum of probability 
day_sum_prob = probabilities_df.sum()
print(day_sum_prob)
df_day_sum_prob = pd.DataFrame(day_sum_prob)


Friday       239.16
Monday       230.27
Saturday     290.02
Sunday       316.18
Thursday     202.49
Tuesday      223.63
Wednesday    211.25
dtype: float64


In [181]:
df_day_sum_prob.reset_index()

,index,0
0,Friday,239.16
1,Monday,230.27
2,Saturday,290.02
3,Sunday,316.18
4,Thursday,202.49
5,Tuesday,223.63
6,Wednesday,211.25


In [185]:
#scale it down with min_max
min_max_scaler = MinMaxScaler()
scaled_prob = min_max_scaler.fit_transform(df_day_sum_prob)
scaled_df = pd.DataFrame(scaled_prob,columns=df_day_sum_prob.columns)
scaled_df


,0
0,0.322544
1,0.244349
2,0.769901
3,1.000000
4,0.000000
5,0.185944
6,0.077052


From the above, I can deduce that crime would most likely happen on the weekends with a focus on sunday then saturday. 